In [1]:
#!pip install numpy pandas xpress PySCIPopt matplotlib networkx

In [2]:
import numpy as np
import pandas as pd
from pyscipopt import Model, quicksum
from read_data import process_all_instances
import itertools

In [3]:
all_instance_data = process_all_instances("./final_test_set")

Processing instance: d76179d0-6e38-4136-a19c-f724a88ea5d4
Processing instance: e5a70e91-6871-45ac-b155-beb01314ecf0
Processing instance: e0bd643d-6cc5-42cf-91d9-34355740a96f
Processing instance: f6da3aa1-6311-4aa5-9d09-077627c66d2c
Processing instance: f99f3fd8-76b5-4385-bf70-8eae14ac9bbf
Processing instance: cd0ee34d-7f72-47a8-8c3d-7c6989803364
Processing instance: de5a9fc8-27b9-4f9b-8545-5c99251c14c3
Processing instance: d1a8c631-9e97-44ad-8d8f-793b71ba704c
Processing instance: d7eab0b2-a1b6-417e-a57f-360d61203014
Processing instance: c369f40a-d875-4bff-b605-2f7e815799df
Processing instance: eec638a3-4c9a-42f0-b551-0397ed9d7d9e
Processing instance: c6fa2c03-39b4-4e04-bdfc-635fbf4d4358
Processing instance: df055c5e-7f11-42cb-b608-6509bca739e2
Processing instance: edb1a32f-b966-4a59-a85c-58d4a2fa1daf
Processing instance: dac8a39f-cbdc-418d-81ec-7b3d80154b1d
Processing instance: d83b88c1-8bb7-481d-be5f-7e14c753bd84
Processing instance: d651785a-8c8f-423b-9b4a-902838e7097a
Processing ins

In [80]:
for i, inst in enumerate(all_instance_data):
    if inst["instance_name"] == "bf69d1de-260c-4af9-bc57-294b8bc8ee5c":
        print(i)

49


In [81]:
keys = list(all_instance_data[0].keys())
# one courier 92
# 2 couiries 14
# 4 couirires 17
# 28

instance_id = 49


instance = all_instance_data[instance_id]

name = instance[keys[0]]

couriers = instance[keys[1]]

orders = instance[keys[2]]

travel_time = instance[keys[3]]

print(name)
print(f"N couriers: {len(couriers)}")
print(f"N Jobs: {len(orders)}")

bf69d1de-260c-4af9-bc57-294b8bc8ee5c
N couriers: 4
N Jobs: 9


In [82]:
def arcs_to_adjacency_matrix(arcs):
    """
    Convert a list of arcs (edges) into an adjacency matrix.

    :param arcs: List of tuples representing arcs (start, end).
    :return: Adjacency matrix as a 2D list or numpy array.
    """
    # Find the set of unique nodes
    nodes = set()
    for arc in arcs:
        nodes.add(arc[0])
        nodes.add(arc[1])

    # Sort nodes and create a mapping from node to matrix index
    sorted_nodes = sorted(nodes)
    node_to_index = {node: idx for idx, node in enumerate(sorted_nodes)}

    # Initialize the adjacency matrix with zeros
    n = len(sorted_nodes)
    adjacency_matrix = np.zeros((n, n), dtype=int)

    # Populate the adjacency matrix based on the arcs
    for arc in arcs:
        start, end = arc
        i = node_to_index[start]
        j = node_to_index[end]
        adjacency_matrix[i][
            j
        ] = 1  # You can replace 1 with the weight of the arc if necessary

    return adjacency_matrix, sorted_nodes


# Example usage

In [83]:
COURIERS = [i for i in range(len(couriers))]

# determine the depos and Capacities
DEPOS = []
CAPACITIES = []
for i in COURIERS:
    DEPOS.append(couriers[i].location)
    CAPACITIES.append(couriers[i].capacity)
MAXCAP = max(CAPACITIES)
# Determine job data
ORDERS = [i for i in range(len(orders))]
PICKUPS = []
DROPOFFS = []
SIZES = {}
TWSTART = []
IDS = []
for i in ORDERS:
    IDS.append(orders[i].delivery_id)
    PICKUPS.append(orders[i].pickup_loc)
    DROPOFFS.append(orders[i].dropoff_loc)
    SIZES[orders[i].pickup_loc] = orders[i].capacity
    SIZES[orders[i].dropoff_loc] = -orders[i].capacity
    TWSTART.append(orders[i].time_window_start)


VERTICES = list(set(DEPOS + PICKUPS + DROPOFFS))

A_1 = list(itertools.product(DEPOS, PICKUPS))
A_2 = [
    (u, v)
    for u in list(set(PICKUPS + DROPOFFS))
    for v in list(set(PICKUPS + DROPOFFS))
    if u != v
]
A_3 = list(itertools.product(DROPOFFS, DEPOS))
A_4 = [(u, u) for u in DEPOS]
A = list(set(A_1 + A_2 + A_3 + A_4))
ARCS, NODES = arcs_to_adjacency_matrix(A)

DISTANCES = np.array(travel_time)[1:, 1:].astype(np.int16)

# defined as a sum all of travel time - just an arbitrary large number
T = np.sum(DISTANCES.flatten())

if np.count_nonzero(DISTANCES) != DISTANCES.shape[0] ** 2 - DISTANCES.shape[0]:
    print("Not fully connected matrix")

In [84]:
def check_triangular_inequality(matrix):
    """
    Check if the triangular inequality is violated in a distance matrix.

    :param matrix: A 2D list or numpy array representing a distance matrix,
                   where matrix[i][j] is the distance between points i and j.
    :return: A list of violated inequalities as tuples (i, j, k)
    """
    n = len(matrix)
    violations = []

    # Iterate over all triplets (i, j, k) to check triangular inequality
    for i in range(n):
        for j in range(n):
            for k in range(n):
                # Check if the inequality d(i, k) <= d(i, j) + d(j, k) is violated
                if matrix[i][k] > matrix[i][j] + matrix[j][k]:
                    violations.append((i, j, k))

    return violations


violations = check_triangular_inequality(DISTANCES)
if violations:
    print("Triangular inequality violated for the following triplets:")
    for i, j, k in violations:
        print(
            f"d({i}, {k})= {DISTANCES[i][k]} > d({i}, {j})={DISTANCES[i][j]}  + d({j}, {k})={DISTANCES[j][k]} "
        )
else:
    print("No violations of the triangular inequality found.")

Triangular inequality violated for the following triplets:
d(1, 10)= 14 > d(1, 2)=3  + d(2, 10)=10 
d(1, 3)= 12 > d(1, 7)=3  + d(7, 3)=8 
d(1, 10)= 14 > d(1, 9)=6  + d(9, 10)=7 
d(1, 10)= 14 > d(1, 14)=6  + d(14, 10)=7 
d(2, 13)= 10 > d(2, 9)=5  + d(9, 13)=4 
d(3, 1)= 12 > d(3, 7)=8  + d(7, 1)=3 
d(3, 4)= 13 > d(3, 7)=8  + d(7, 4)=4 
d(4, 14)= 9 > d(4, 1)=2  + d(1, 14)=6 
d(4, 10)= 15 > d(4, 2)=4  + d(2, 10)=10 
d(4, 3)= 13 > d(4, 7)=4  + d(7, 3)=8 
d(6, 16)= 9 > d(6, 7)=6  + d(7, 16)=2 
d(8, 9)= 14 > d(8, 1)=7  + d(1, 9)=6 
d(8, 9)= 14 > d(8, 2)=8  + d(2, 9)=5 
d(8, 9)= 14 > d(8, 5)=8  + d(5, 9)=5 
d(9, 8)= 14 > d(9, 1)=6  + d(1, 8)=7 
d(9, 17)= 12 > d(9, 1)=6  + d(1, 17)=5 
d(9, 8)= 14 > d(9, 2)=5  + d(2, 8)=8 
d(9, 17)= 12 > d(9, 2)=5  + d(2, 17)=6 
d(9, 8)= 14 > d(9, 5)=5  + d(5, 8)=8 
d(9, 17)= 12 > d(9, 5)=5  + d(5, 17)=6 
d(10, 1)= 14 > d(10, 2)=10  + d(2, 1)=3 
d(10, 4)= 15 > d(10, 2)=10  + d(2, 4)=4 
d(10, 1)= 14 > d(10, 9)=7  + d(9, 1)=6 
d(10, 1)= 14 > d(10, 14)=7  + d(14, 1

In [85]:
DISTANCES

array([[ 0, 13, 10, 10, 14, 10, 14, 10, 15,  7,  1, 10,  7,  4,  7,  6,
        10, 13],
       [13,  0,  3, 12,  2,  3,  6,  3,  7,  6, 14, 11,  6, 10,  6,  7,
         4,  5],
       [10,  3,  0,  9,  4,  1,  7,  1,  8,  5, 10,  8,  5, 10,  6,  7,
         2,  6],
       [10, 12,  9,  0, 13,  9,  9,  8, 10,  8, 10,  1,  7, 11,  8,  9,
         8,  8],
       [14,  2,  4, 13,  0,  4,  4,  4,  6,  8, 15, 12,  8, 12,  9,  9,
         5,  5],
       [10,  3,  1,  9,  4,  0,  7,  1,  8,  5, 11,  9,  5,  9,  6,  7,
         2,  6],
       [14,  6,  7,  9,  4,  7,  0,  6,  2, 12, 15,  8, 11, 16, 12, 13,
         9,  1],
       [10,  3,  1,  8,  4,  1,  6,  0,  8,  6, 11,  9,  5, 10,  6,  7,
         2,  6],
       [15,  7,  8, 10,  6,  8,  2,  8,  0, 14, 16,  9, 13, 17, 13, 14,
        10,  2],
       [ 7,  6,  5,  8,  8,  5, 12,  6, 14,  0,  7,  8,  1,  4,  1,  2,
         4, 12],
       [ 1, 14, 10, 10, 15, 11, 15, 11, 16,  7,  0, 10,  8,  4,  7,  7,
        10, 14],
       [10, 11,  8,  

In [86]:
# let us model the problem with scip

from pyscipopt import Model, quicksum, SCIP_PARAMSETTING
import itertools

model = Model()

t = {}
for i in VERTICES:
    t[i] = model.addVar(vtype="C", name=f"t[{i}]", ub=180)

q = {}
for i in VERTICES:
    q[i] = model.addVar(vtype="C", name=f"q[{i}]", ub=MAXCAP)

x = {}
for i in VERTICES:
    for j in VERTICES:
        for k in COURIERS:

            x[i, j, k] = model.addVar(vtype="B", name=f"x[{i},{j},{k}]")


# 2
# every vertex should be visited only ones
for v in VERTICES:
    model.addCons(
        quicksum(
            x[u, v, k] for u in VERTICES for k in COURIERS if ARCS[u - 1][v - 1] == 1
        )
        == 1,
        name="2_{v}",
    )

# 3
# the same vehicle that visits a vertex does also leave it
for v in VERTICES:
    for k in COURIERS:
        model.addCons(
            quicksum(x[v, u, k] for u in VERTICES if ARCS[v - 1][u - 1] == 1)
            == quicksum(x[u, v, k] for u in VERTICES if ARCS[u - 1][v - 1] == 1),
            name="3_{v}_{k}",
        )

# 4
# guarantee that the same vehicle that visits a
# pickup vertex does also visit the corresponding delivery vertex
for k in COURIERS:
    for o in ORDERS:
        model.addCons(
            quicksum(
                x[u, PICKUPS[o], k]
                for u in VERTICES
                if ARCS[u - 1][PICKUPS[o] - 1] == 1
            )
            == quicksum(
                x[u, DROPOFFS[o], k]
                for u in VERTICES
                if ARCS[u - 1][DROPOFFS[o] - 1] == 1
            )
        )

# 5
# only the vehicle corresponding to its depot can leave it
for k in COURIERS:
    model.addCons(
        quicksum(x[DEPOS[k], v, k] for v in VERTICES if ARCS[DEPOS[k] - 1][v - 1]) == 1
    )

# 6
# ensure that the time variables for the vertices are correctly updated
# A1 from depos to pickup
# A2 form pickup to dropoff (u!=v)

for k in COURIERS:
    for arc in list(set(A_1 + A_2)):
        u, v = arc[0], arc[1]
        # print(f'{t[u]} + ({DISTANCES[u - 1, v - 1]}+{T})* {x[u, v, k]}<=({t[v]}+{T})')
        model.addCons(t[u] + (DISTANCES[u - 1, v - 1] + T) * x[u, v, k] <= t[v] + T)


# 7
# ensure that the pickup is visited before the corresponding dropoff.
for o in ORDERS:
    # print(t[DROPOFFS[o]],t[PICKUPS[o]],DISTANCES[DROPOFFS[o] - 1, PICKUPS[o] - 1])
    model.addCons(
        t[DROPOFFS[o]] - t[PICKUPS[o]] >= DISTANCES[PICKUPS[o] - 1, DROPOFFS[o] - 1]
    )

# 8

# A3 from dropoff to depos
# A4 loops on starts

for k in COURIERS:
    for arc in list(set(A_1 + A_2)):
        u, v = arc[0], arc[1]
        # print(f'{q[u]} + ({SIZES[v]}+{MAXCAP})* {x[u, v, k]}<=({q[v]}+{MAXCAP})')
        model.addCons(q[u] + (SIZES[v] + MAXCAP) * x[u, v, k] <= (q[v] + MAXCAP))

# 9
# load at the depots is already fixed
for k in COURIERS:
    model.addCons(q[DEPOS[k]] == (MAXCAP - CAPACITIES[k]))

# 10
# t is representing the time when a vertex is visited by a vehicle. As we can see, this
# variable has to respect the bounds of the respective time window.

for o in ORDERS:
    # print(t[PICKUPS[o]],TWSTART[o])
    model.addCons(t[PICKUPS[o]] >= TWSTART[o])

# 11
# new constraint with prohibiting more than 4 jobs
for k in COURIERS:
    model.addCons(
        quicksum(x[u, v, k] for u, v in list(itertools.product(PICKUPS, DROPOFFS))) <= 4
    )

# define objective
model.setObjective(quicksum(t[v] for v in DROPOFFS))

# model.setParam("presolving/maxrounds", 0)
# model.setParam("presolving/maxrestarts", 0)

# model.setHeuristics(SCIP_PARAMSETTING.AGGRESSIVE)
model.setParam("limits/time", 1200)
model.optimize()
model.writeProblem("test.lp")
if model.getNSols() == 0:
    print("No feasible solution found.")
else:
    print(f"Objective value={model.getObjVal()}")
    print(f"Solution: x={model.getBestSol()}")

presolving:
(round 1, fast)       336 del vars, 13 del conss, 0 add conss, 21 chg bounds, 808 chg sides, 808 chg coeffs, 0 upgd conss, 0 impls, 22 clqs
(round 2, fast)       336 del vars, 85 del conss, 0 add conss, 31 chg bounds, 1624 chg sides, 1624 chg coeffs, 0 upgd conss, 0 impls, 22 clqs
(round 3, fast)       336 del vars, 85 del conss, 0 add conss, 31 chg bounds, 2108 chg sides, 2108 chg coeffs, 0 upgd conss, 0 impls, 22 clqs
(round 4, exhaustive) 336 del vars, 85 del conss, 0 add conss, 38 chg bounds, 2472 chg sides, 2472 chg coeffs, 0 upgd conss, 0 impls, 22 clqs
(round 5, exhaustive) 336 del vars, 85 del conss, 0 add conss, 38 chg bounds, 2472 chg sides, 2472 chg coeffs, 114 upgd conss, 0 impls, 22 clqs
(round 6, fast)       344 del vars, 85 del conss, 0 add conss, 38 chg bounds, 2472 chg sides, 2472 chg coeffs, 114 upgd conss, 88 impls, 22 clqs
(round 7, medium)     348 del vars, 85 del conss, 0 add conss, 38 chg bounds, 2476 chg sides, 2476 chg coeffs, 114 upgd conss, 88 imp

830000e+02 | 2.030000e+02 |  10.93%|  34.29%
 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
 27.3s|   500 |    52 | 46987 |  86.6 |   149M |  24 | 784 |2373 |1583 |6664 |  4 |1394 |2522 | 1.830000e+02 | 2.030000e+02 |  10.93%|  36.77%
 32.5s|   600 |    52 | 62475 |  98.0 |   156M |  26 | 784 |2276 |   0 |8112 |  0 |1612 |2887 | 1.830000e+02 | 2.030000e+02 |  10.93%|  38.89%
 41.0s|   700 |    55 | 94526 | 129.8 |   207M |  30 | 784 |2357 |1696 |  10k| 15 |1850 |3203 | 1.830000e+02 | 2.030000e+02 |  10.93%|  43.37%
 71.6s|   800 |    59 |340037 | 420.9 |   216M |  30 | 784 |2432 |1624 |  11k|  2 |2140 |3580 | 1.830000e+02 | 2.030000e+02 |  10.93%|  46.27%
 75.1s|   900 |    50 |355476 | 391.2 |   217M |  30 | 784 |2522 |1657 |  12k|  0 |2280 |3784 | 1.830000e+02 | 2.030000e+02 |  10.93%|  46.61%
 80.4s|  1000 |    36 |404657 | 401.3 |   218M |  30 | 784 |2548 |   0 |  12k|  0 |2370 |3828 | 1

In [87]:
def chain_arcs(arcs):
    if not arcs:
        return []

    # Build a dictionary to quickly find arcs by their start point
    start_to_arc = {arc[0]: arc for arc in arcs}

    # Choose an arbitrary starting arc
    start_arc = arcs[0]

    # Create the ordered list of arcs
    ordered_arcs = [start_arc]
    current_arc = start_arc

    # Chain the arcs by matching the end of the current arc to the start of the next
    while len(ordered_arcs) < len(arcs):
        # Find the next arc that starts where the current arc ends
        next_arc = start_to_arc.get(current_arc[1])
        if next_arc is None:
            raise ValueError("No matching arc found to continue the chain.")
        ordered_arcs.append(next_arc)
        current_arc = next_arc

    # Ensure the last arc connects to the first one for circularity
    if ordered_arcs[-1][1] != ordered_arcs[0][0]:
        raise ValueError("Arcs do not form a complete circular chain.")

    return ordered_arcs


# Example usage
# edges2= chain_arcs([(1, 5), (2, 3),(3, 4), (4, 1), (5, 6), (6, 2)])
# print(edges2)

In [88]:
sol = model.getBestSol()
routes = []
for k in COURIERS:
    edges = []
    for i in VERTICES:
        for j in VERTICES:
            if model.getVal(x[i, j, k]) > 0.5:  # variable is binary so > 0.5 --> is 1
                edges.append((i, j))
    if len(edges) > 1:

        edges = chain_arcs(edges)
    routes.append((k + 1, edges))
print(model.getObjVal())
print("SOLUTION")
print(routes)

203.0
SOLUTION
[(1, [(1, 10), (10, 14), (14, 11), (11, 1)]), (2, [(2, 15), (15, 16), (16, 17), (17, 2)]), (3, [(3, 5), (5, 6), (6, 7), (7, 3)]), (4, [(4, 8), (8, 9), (9, 12), (12, 13), (13, 18), (18, 4)])]


In [89]:
couriers

[Courier(ID=1, Location=1, Capacity=100),
 Courier(ID=2, Location=2, Capacity=100),
 Courier(ID=3, Location=3, Capacity=100),
 Courier(ID=4, Location=4, Capacity=100)]

In [90]:
orders

[Delivery(ID=5, Capacity=17, Pickup Loc=5, Time Window Start=10, Pickup Stacking Id=108056, Dropoff Loc=6),
 Delivery(ID=6, Capacity=24, Pickup Loc=5, Time Window Start=11, Pickup Stacking Id=108056, Dropoff Loc=7),
 Delivery(ID=7, Capacity=3, Pickup Loc=8, Time Window Start=10, Pickup Stacking Id=151391, Dropoff Loc=9),
 Delivery(ID=8, Capacity=7, Pickup Loc=10, Time Window Start=13, Pickup Stacking Id=147289, Dropoff Loc=11),
 Delivery(ID=9, Capacity=4, Pickup Loc=12, Time Window Start=23, Pickup Stacking Id=132201, Dropoff Loc=13),
 Delivery(ID=10, Capacity=15, Pickup Loc=10, Time Window Start=16, Pickup Stacking Id=147289, Dropoff Loc=14),
 Delivery(ID=11, Capacity=14, Pickup Loc=15, Time Window Start=8, Pickup Stacking Id=98018, Dropoff Loc=16),
 Delivery(ID=12, Capacity=12, Pickup Loc=15, Time Window Start=7, Pickup Stacking Id=98018, Dropoff Loc=17),
 Delivery(ID=13, Capacity=15, Pickup Loc=12, Time Window Start=28, Pickup Stacking Id=132201, Dropoff Loc=18)]

In [123]:
DISTANCES

array([[ 0, 10, 12,  8,  8, 23],
       [10,  0, 10, 17, 18, 33],
       [12, 10,  0, 20, 20, 34],
       [ 8, 17, 20,  0,  6, 14],
       [ 8, 18, 20,  6,  0, 15],
       [23, 33, 34, 14, 15,  0]], dtype=int16)